## Corrercciones futuras
- El modelo va a predecir una cantidad `N` de dias. No recibe nada mas
- A futuro el modelo se mueve con el tiempo.

## Consideraciones
- Supuesto: Inicialmente, el ultimo dia a la fecha es (el ultimo dia del dataset)
- Dependiendo del ultimo dia con el cual se entreno se calculan `N` dias mas y se crea ese rango de fechas para crear las variables exogenas

In [32]:
import numpy as np
import pandas as pd

# Opcion 1, lo hacemos desde 0 con grid_search
import itertools
from sklearn.model_selection import TimeSeriesSplit
#import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from utility_funcs import *


import joblib

In [33]:
# Cargue de datos
demanda_df = pd.read_csv(".\Datos\demanda.csv")

In [34]:
demanda_prod_df = demanda_df[demanda_df['id_producto']==1]
demanda_prod_df['date'] = pd.to_datetime(demanda_prod_df['date'], format='%Y-%m-%d')

C:\Users\juanc\AppData\Local\Temp\ipykernel_21632\3668112741.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demanda_prod_df['date'] = pd.to_datetime(demanda_prod_df['date'], format='%Y-%m-%d')


In [35]:
# Primero, asegurar que todas las fechas estan
# Los datos de demanda iniciales no tienen enero 2020
fecha_inicial = pd.to_datetime('2020-01-01')
fechas_df = pd.DataFrame({'date': pd.date_range(start=fecha_inicial, end=demanda_prod_df['date'].max())})
complete_demanda_prod_df = fechas_df.merge(demanda_prod_df
                                           ,how='left'
                                           ,left_on='date'
                                           ,right_on='date'
                                           )

In [36]:
exog_dict = create_exog_cols_from_date_series(complete_demanda_prod_df['date']
                                              , event_date='2021-07-02'
                                              , event_name='llegada_tienda_comp'
                                              )
exog_df = pd.DataFrame(exog_dict)

In [37]:
assert complete_demanda_prod_df.shape[0] == exog_df.shape[0]

data_df = pd.concat([complete_demanda_prod_df, exog_df], axis=1)

## No se realiza ningun preprocesamiento para pasar al despliegue del modelo

In [38]:
# Entrenamiento de modelo ARIMA - Suponemos que mejores parametros son (2,1,7)
N_train = 400 # cantidad de dias previos usados en el entrenamiento
y_train = data_df['demanda'][-N_train:]
X_exog = data_df[exog_df.columns]
X_exog_train = X_exog.iloc[-N_train:, :]

In [39]:
modelo_arima = ARIMA(y_train, order=(2, 1, 7)
                     , enforce_stationarity=False
                    , enforce_invertibility=False
                    , exog=X_exog_train.astype(float)
                    )
modelo_fit = modelo_arima.fit(method_kwargs={"maxiter": 200})

# Predicciones en el conjunto de prueba
# predictions_arima = modelo_fit.forecast(steps=len(y_test), exog=X_exog_test)

c:\Users\juanc\Desktop\AI-DemandForecast\ai-forecast\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [40]:
# Ultima fecha de entrenamiento para guardar como metadato
ultima_fecha = list(data_df['date'][-N_train:])[-1]
ultima_fecha_dict = {'ultima_fecha': ultima_fecha.strftime('%Y-%m-%d')}

In [41]:
import json
import pickle

In [42]:
# Output de modelo en formato Joblib conservando informacion adicional de las fechas de entrenamiento
model_name = "forecast_model_with_exogenous-0.1.0.pkl"
#joblib.dump(modelo_arima, model_name)
# Guarda el modelo en un archivo
with open('./' + model_name, 'wb') as f:
    pickle.dump(modelo_fit, f)

with open('metadata.json', 'w') as f:
    json.dump(ultima_fecha_dict, f)